In [58]:
import numpy as np
import pandas as pd
import re

from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_score

In [2]:
PATH_tweets = '../Data/tweets.csv'

In [3]:
tweets = pd.read_csv(
    PATH_tweets,
    usecols=['id', 'screen_name', 'text'], 
    index_col='id', 
    dtype={'screen_name': str, 'text': str}
    )
tweets = tweets.drop_duplicates(subset='text', keep=False)
tweets = tweets.dropna()

In [4]:
def limpiar_retweet(tweet:str):
    if tweet.startswith('RT'):
        tweet = ''.join(tweet.split(': ')[1:])
    return tweet

tweets['text'] = tweets['text'].apply(limpiar_retweet)

In [5]:
tweets.loc[:, 'text'] = tweets['text'].str\
        .replace('[.,]', '', regex=True).str.lower()\
        .apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))\
        .str.split().apply(lambda x: np.array(x))

In [6]:
def shingles(k, words: np.ndarray):
    shingles = []
    for i in range(0, len(words)):
        shingles.append(' '.join(words[i:i+k]))
    return list(set(shingles))

In [7]:
tweets['shingles'] = tweets['text'].apply(lambda x: shingles(2, x))

In [23]:
shings = tweets['shingles'].to_numpy()
mlb = MultiLabelBinarizer(sparse_output=True)
caracteristica = mlb.fit_transform(shings)

In [66]:
jaccard_score(caracteristica[0][0], caracteristica[1][0], average='micro')

0.0